<h1><span style="background-color:blue; color:white; padding: 5px;">Entrainment de modèles avec RandomForest Regressor<br></h1>
</h4><span style="background-color:red; color:white; padding: 5px;">Utiliser un outil permettant de rafraîchir la page régulièrement pour éviter la mise en sommeil de MLflow sur Heroku
</span></h4>

In [1]:
import pandas as pd
from functions2 import *

2024-02-17 15:06:21.233462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:06:21.653582: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:06:21.659632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 15:06:23.130683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<h2><span style="background-color:grey; color:white; padding: 5px;">Choix des paramètres pour l'entraînement, le stockage sur MLflow et des données à récupérer.</span></h2>

In [2]:
#//////////////////////////////////////////////////////////////////////////////
#                       Option de filtrage des données
#//////////////////////////////////////////////////////////////////////////////
# Variable permettant de choisir si on veut traiter les outliers
gestion_outliers =True

# Nombre de ventes minimum sur une commune pour calculer un prix moyen au m² par commune
nb_ventes_mini = 10

# Taux de filtrage : 
# - Le premier élément est le nombre de vente minimum par commune
# - Le deuxième élément est le pourcentage de outliers dans une commune
#
# Exemple pour tx_filtrage =[10,30] les lignes qui seront conservées seront :
# Les communes dans lesquelles il restera minimum 10 ventes après suppression des outliers
# ET  moins de 30% d'outliers
# ET les lignes inférieures à la limite des outliers
# tx_filtrage = [0,100] => Suppression des lignes dont le montant est au dessus de la limite des outliers.
tx_filtrage =[nb_ventes_mini,30]

# Recherche sur une région spécifique si recherche sur toute la France mettre ''
region=''

# Recherche sur un type de bien spécifique si recherche sur tous les bien mettre ''
type_de_bien =''

# Inclure la surface du terrain True ou False
surface_terrain = True

# Nombre de mois pour l'entraînement et le test (train=80% test= 20%)
# Exemple si 15 mois environ 12 mois pour l'entraînement et 3 mois pour le test
# Mettre None pour utiliser toutes les données disponible
nb_mois=30

#*******************  Pour ajouter d'autres variables :  ***********************
#*********  modifier la requête dans la fonction construcion_requete() ********* 

#//////////////////////////////////////////////////////////////////////////////
#                       Configuration de GridSearchCV
#//////////////////////////////////////////////////////////////////////////////
param_grid = {
    'n_estimators': [50,100],
    'max_depth': [10,20],
    'min_samples_split': [2,5]
    }   
cv=5

#//////////////////////////////////////////////////////////////////////////////
#                       Configuration de mlfow
#//////////////////////////////////////////////////////////////////////////////
# Configuration de mlflow
uri_tracking = "https://mlflowimmoappkevleg-737621d410d0.herokuapp.com/"
experiment_name = "RandomForestRegressor"

# runs à effectuer all_datas 15months all_datas_surface_terrain 15months_surface_terrain)
# run_name = f"{region}_{'Appartement_Maison' if len(type_de_bien)==0 else type_de_bien}"\
#     f"_{nb_mois}months_GridSearch_GestionOutliers_{'oui' if gestion_outliers==True else 'non'}"\
#     f"_{'' if surface_terrain==False else 'surface_terrain'}"
run_name ="NameRegion_NameTypeBien_Appt_Maison_GestionOutliersOui_SurfaceTerrainOui_30Mois"
model_name = f"RFR_{run_name}"

<h2><span style="background-color:grey; color:white; padding: 5px;">Création de la requête </span></h2>

In [3]:
where_clause, query = construcion_requete(region,
                        type_de_bien,
                        nb_ventes_mini,
                        surface_terrain,
                        nb_mois)
print(query)


    # Table pour compter le nombre de ventes par commune
    WITH nb_ventes_mini AS(
    SELECT
        ID_COMMUNE AS ID_COMMUNE,
        count(*) nb_ventes_par_commune
    FROM VENTES 
    WHERE DATE_MUTATION >= DATE_SUB((SELECT MAX(DATE_MUTATION) FROM VENTES), INTERVAL 30 MONTH)
    GROUP BY ID_COMMUNE
    )

    # Selection des variables voulues pour l'entraînement du modèle
    SELECT 
        V.SURFACE_BATI,
        V.ID_COMMUNE,
        V.DATE_MUTATION,
        T.NAME_TYPE_BIEN,
        R.Name_region,
        V.SURFACE_TERRAIN,
        V.MONTANT
    FROM VENTES V
    INNER JOIN TYPES_BIENS as T ON V.ID_TYPE_BIEN = T.ID_TYPE_BIEN
    INNER JOIN COMMUNES AS C ON V.ID_COMMUNE = C.ID_COMMUNE
    INNER JOIN DEPARTEMENTS AS D ON C.ID_DEPT = D.ID_DEPT
    INNER JOIN REGIONS R ON D.ID_REGION = R.ID_REGION
    WHERE R.Name_region NOT IN('Martinique', 'Guyane', 'La Réunion', 'Mayotte', 'Guadeloupe')  AND V.DATE_MUTATION >= DATE_SUB((SELECT MAX(DATE_MUTATION) FROM VENTES), INTERVAL 30 MONT

<h2><span style="background-color:grey; color:white; padding: 5px;">Chargement des données et suppression des outliers. </span></h2>

In [4]:
df = loading_data(query)

if gestion_outliers==True:
    df=filtrage(df,tx_filtrage,where_clause)

Chargement des données en cours...
Création engine sqlalchemy OK


Chargement des données ok
Il y a 23205 communes avec plus de 10 ventes avant suppression des outliers, pour un total de 2406641 ventes
Chargement des données en cours...
Création engine sqlalchemy OK
Chargement des données ok


,ID_COMMUNE,nb_outliers,total_ventes,ventes_restantes,NAME_COMMUNE,pourcentage_ventes_retirees
0,33063,2591,14245,11654,Bordeaux,18.19
1,92012,2185,5056,2871,Boulogne-Billancourt,43.22
2,06088,2105,22827,20722,Nice,9.22
3,92051,1969,2731,762,Neuilly-sur-Seine,72.10
4,44109,1620,16226,14606,Nantes,9.98
5,31555,1481,24005,22524,Toulouse,6.17
6,06029,1466,7787,6321,Cannes,18.83
7,92044,1277,2914,1637,Levallois-Perret,43.82
8,78646,1231,3017,1786,Versailles,40.80
9,94068,1200,3524,2324,Saint-Maur-des-Fossés,34.05


Nombre d'outliers : 142947
Nombre de commune avec des outliers : 8574
Nombre de communes contenant des outliers et pour lesquels ilreste plus de 10 ventes après suppression des outliers : 7811
Nombre de communes avec plus de 30% de ventes retirées : 508
Nombre de communes qui seront retirées : 1105
Nombre de ventes restantes après suppression des outlier et après filtrage : 2205382
Il y a donc eu 8.36% de lignes supprimées après filtrage


<h2><span style="background-color:grey; color:white; padding: 5px;">Entraînement du modèle et stockage des données avec MLflow. </span></h2>

In [5]:
X_train,y_train, X_test, y_test = split_with_m2(df)

X_train,X_test,encoders,scalers =  encod_scal(X_train,X_test)

model, best_params = train_model_randomforest(X_train,y_train, param_grid, cv)

# Tracé et enregistrement du graphique permettant de savoir si le modèle contient suffisamment de données.
images = plot_validation_learning_curve(model, X_train, y_train)

param_mlflow(uri_tracking=uri_tracking,
        experiment_name=experiment_name, run_name=run_name,
        best_params=best_params,
        model=model, model_name=model_name,
        X_test=X_test, y_test=y_test,
        encoders=encoders,scalers=scalers,
        images = images).log_mlflow()

Split des données en cours...


/home/kevin/workspace/PCO/certif_app_immo/model/functions2.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['M2'] = df_train['MONTANT'] / df_train['SURFACE_BATI']


Split OK
Normalisation des données en cours...
Normalisation des données OK
Entraînement en cours ...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time= 2.7min
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time= 3.0min
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time= 2.5min
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time= 2.4min
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time= 2.2min
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time= 5.2min
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time= 5.1min
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time= 5.0min
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time= 5.3min
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time= 5.3min
[CV] END .max_depth=10, min_

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_mae,split1_test_mae,...,std_test_mae,rank_test_mae,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,153.089401,15.360142,1.415070,0.184803,10,2,50,"{'max_depth': 10, 'min_samples_split': 2, 'n_e...",-45663.615047,-43297.756788,...,944.839350,4,0.674863,0.670609,0.672456,0.668214,0.624905,0.662209,0.018780,4
1,308.738683,5.931081,2.684948,0.234375,10,2,100,"{'max_depth': 10, 'min_samples_split': 2, 'n_e...",-45658.206608,-43298.002859,...,945.622370,3,0.675054,0.670649,0.672547,0.668384,0.624915,0.662310,0.018826,3
2,169.900941,14.742073,1.529962,0.266720,10,5,50,"{'max_depth': 10, 'min_samples_split': 5, 'n_e...",-45677.308364,-43287.750152,...,949.900133,2,0.674771,0.670845,0.672640,0.668388,0.624969,0.662323,0.018794,2
3,346.992437,12.754522,3.080674,0.260333,10,5,100,"{'max_depth': 10, 'min_samples_split': 5, 'n_e...",-45657.642598,-43284.856922,...,949.739342,1,0.675088,0.670823,0.672517,0.668420,0.624960,0.662362,0.018827,1
4,293.523154,26.189132,5.972636,1.504375,20,2,50,"{'max_depth': 20, 'min_samples_split': 2, 'n_e...",-47372.494773,-44406.570703,...,1042.111403,8,0.648784,0.654204,0.658089,0.658129,0.612813,0.646404,0.017140,8


Ré-entraînement avec les meilleurs hyperparamètres en cours...
